# Vision Transformer

![Vision Transformer Model](visiontransformer1.png)

Bu notebookta sizlere bir Vision Transformer (ViT) modeli implement edeceğiz. Transformer mimarisi, Ashish Vaswani ve diğerleri tarafından yazılan [Vision Transformer (ViT)](https://arxiv.org/abs/2010.11929) makalesine dayanmaktadır. Model, [PyTorch](https://pytorch.org/) kullanılarak implement edilmiştir.

 ### 1. Kütüphaneleri içe aktarma

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


 ### 2. Hiperparametreleri Tanımlama

In [2]:
patch_size = 16 # Fotoğrafı böleceğimiz parçaların boyutu 
num_layers = 12 # Encoder katman sayısı
embedding_dim = 768 # Embedding boyutu
num_heads = 12 # MSA başlığı sayısı
forward_expansion = 3  # Encoder içindeki ileri beslemeli ağ nöron artış oranı
dropout_rate = 0.1 # Dropout oranı
attn_dropout = 0.1 # Attention Dropout oranı
image_channels = 3 # Fotoğrafın kanal sayısı 
image_size = 32 # Fotoğrafın boyutu

### 3. Vision Transformer (ViT) Modeli Blokları

### Aşamalar

##### 3.1 Embeddings
- **3.1.1.** Patch Embeddings
- **3.1.2.** Patch+Pos Embeddings

##### 3.2. Encoder
- **3.2.1.** Multi Head Attention
- **3.2.2.** Encoder Stack
- **3.2.3.** Encoder

##### 3.3. MLP Head
- **3.3.1.** Sınıflandırma Başlığı

##### 3.4. **Vision Transformer**

<br>

### 3.1 Embeddings


#### 3.1.1. **Patch Embeddings**

Görseli Encoder'a vermek için öncelikle onu sıralı bir diziye dönüştürüp ondan embeddingler elde etmemiz gerekir. Bunun için görseli (flattening) düzleştirmek bir seçenek olsa da Encoder içinde gerçekleşecek MSA (Multihead Self Attention) işlemi karesel bir işlem olduğu için çok ciddi bir hesaplama maliyeti gerektirir. Örneğin gerçek hayat için çok düşük bir çözünürlük olan $(224,224)$ boyutundaki bir görüntüyü düzleştirirseniz, $50176$ uzunluğunda bir vektör elde ederiz ki bunu hesaplamak çok ciddi bir maliyet gerektirir. 

Bu sebeple görselleri direkt düzleştirme yerine görseli patch adını verdiğimiz parçalara ayırıyoruz ve ardından bir lineer katmandan geçirerek bunun sonucunda embeddingsleri elde edebiliyoruz. 

Bu işlemleri gerçekleştirmek adına makalede 2 yöntem öneriliyor: Bunlardan ilki görseli $(P, P)$ boyutunda patchlere ayırmak ve ardından bu patchleri $(N, P^2C)$ boyutunda bir matrise dönüştürüp lineer katmadan geçirerek embeddingsleri elde etmek. İkincisi ise kernel ve stride boyutu $(P, P)$ olan bir Evrişimli Sinir Ağı (Convolutional Neural Network) kullandıktan sonra o ağı düzleştirerek embeddingsleri elde etmek.

Her iki yöntem için de gerekli kodu yazdım, ikisini de kullanabilirsiniz


In [14]:
# Using CNN
class PatchEmbedding(nn.Module):
    def __init__(self, image_channels, image_size, patch_size, embedding_dim):
        super().__init__() 
        image_size = image_size
        patch_size = patch_size

        self.cnn_proj = nn.Conv2d(in_channels=image_channels, out_channels=embedding_dim, kernel_size=patch_size, stride=patch_size)
    
    def forward(self,x):
        x = self.cnn_proj(x)
        x = x.flatten(2)
        x = x.transpose(1,2)
        return x

# Using MLP
class PatchEmbedding2(nn.Module):
    def __init__(self, image_channels, image_size, patch_size, embedding_dim):
        super().__init__()

        self.patches = nn.Unfold(kernel_size=patch_size, stride=patch_size)

        self.mlp_proj = nn.Linear(in_features=image_channels*patch_size*patch_size, out_features=embedding_dim)
    
    def forward(self,x):
        x = self.patches(x)
        x = x.transpose(1,2)
        x = self.mlp_proj(x)
        return x

#### 3.1.2. **Patch+Pos Embeddings**

Patch embeddinglere ulaştığımıza göre sıra ona konumsal bilgileri de ekleyerek birleştirmekte. Bunun için de bir pos embedding layer oluşturuyoruz. Bu layer, patch embeddinglerin boyutu kadar bir çıktı üretecek. Bu çıktıyı patch embeddinglerle toplayarak patch+pos embeddingleri elde edeceğiz.

Burada değinmem gereken çok önemli bir konu ise  **class token** adını verdiğimiz bir ek embedding. Bu vektör, modelin hangi sınıfa ait olduğunu bilmesini sağlayacak. Bu embedding'i da patch+pos embeddingslere ekleyeceğiz.


In [15]:
class PatchPosEmbeddings(nn.Module):
    def __init__(self, image_channels, image_size, patch_size, embedding_dim, dropout_rate):
        super().__init__()

        num_patches = (image_size // patch_size) ** 2
        
        self.patch_embeddings = PatchEmbedding(image_channels, image_size, patch_size, embedding_dim)
        self.position_embeddings = nn.Parameter(torch.zeros(1, num_patches + 1, embedding_dim))

        self.cls = nn.Parameter(torch.zeros(1, 1, embedding_dim))

        self.drop = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        b = x.shape[0]
        patch_emb = self.patch_embeddings(x)
        cls_tokens = self.cls.expand(b,-1,-1)
        patch_emb_cls = torch.cat((cls_tokens,patch_emb),dim=1)
        pos_embed = self.position_embeddings
        return self.drop(patch_emb_cls + pos_embed)

### 3.2. Encoder

#### 3.2.1. **Multihead Self Attention**

 Bu bölümde Multihead Self Attention katmanını uygulayacağız. Çoklu kafa dikkat katmanı, $h$ adet dikkat başlığına sahiptir. Her dikkat başlığının ayrı bir sorgu, anahtar ve değer matrisi vardır.

In [ ]:
class MSA(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1, proj_attn=0.2):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads

        # We check if d_model is divisible by num_heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # In order to assign the head size in each head we divide d_model by num_heads. This will be the for both d_k and d_v
        self.d_qkv = d_model // num_heads

        # We use nn.Linear to project the queries, keys, and values. We used the same linear projection for all heads.
        # The reason for this is that it allows us to use a single matrix multiplication to project the queries, keys and values
        # This is more efficient than using separate matrices for each
        self.W_keys = nn.Linear(d_model, d_model)
        self.W_queries = nn.Linear(d_model, d_model)
        self.W_values = nn.Linear(d_model, d_model)

        # We use a single linear projection to project the output of the attention heads
        self.linear_proj = nn.Linear(d_model, d_model)

        self.attn_dropout = nn.Dropout(dropout)
        self.proj_dropout = nn.Dropout(proj_attn) 

    def forward(self, key_src, query_src, value_src):
        
        # We get the shape of the input batch
        B,T,C = key_src.shape # (batch_size, seq_len, d_model)


        # We project the queries, keys and values using their respective weight matrices
        keys = self.W_keys(key_src) # (batch_size, seq_len, d_model)
        queries = self.W_queries(query_src) # (batch_size, seq_len, d_model)
        values = self.W_values(value_src) # (batch_size, seq_len, d_model)
        

        # We reshape the queries, keys and values so that we can split them into multiple heads
        
        keys = keys.view(B,T,self.num_heads,self.d_qkv) # (batch_size, seq_len, num_heads, d_qkv)
        queries = queries.view(B,T,self.num_heads,self.d_qkv) # (batch_size, seq_len, num_heads, d_qkv)
        values = values.view(B,T,self.num_heads,self.d_qkv) # (batch_size, seq_len, num_heads, d_qkv)


        # We transpose the queries, keys and values so that the shape of the tensor becomes (batch_size, num_heads, seq_len, d_qkv)

        keys = keys.permute(0,2,1,3) # (batch_size, num_heads, seq_len, d_qkv)
        queries = queries.permute(0,2,1,3) # (batch_size, num_heads, seq_len, d_qkv)
        values = values.permute(0,2,1,3) # (batch_size, num_heads, seq_len, d_qkv)

        # We compute the attention scores.
        atn_scr = queries @ keys.transpose(-2,-1) # (batch_size, num_heads, seq_len, seq_len)
        # We scale the attention scores
        scaled_atn_scr = atn_scr / self.d_qkv**-0.5
        
        # We apply the softmax activation to compute the attention weights
        attention_weights = torch.softmax(scaled_atn_scr, dim=-1)
        attention_weights = self.attn_dropout(attention_weights)  # Applying dropout
        # Lastly we multiply the attention weights with the values
        out = attention_weights @ values
        out = out.transpose(1, 2)
        # Reshape the matrix to (batch_size, seq_len, d_model) in order to be able to feed it to the next layer
        out = out.reshape(B, T, C)
        # Apply one last linear projection
        out = self.linear_proj(out)
        out = self.proj_dropout(out)

        return out



#### 3.2.2. **Encoder Stack**

Bu bölümde Encoder Stack bloğunu oluşturacağız. Encoder Stack bloğu 1 MSA ve 1 MLP bloğundan oluşur.

Bir sonraki bölümde bu bloğu kullanarak çoklu katmanlı Encoder bloğu oluşturacağız.

In [17]:
class EncoderStack(nn.Module):
    def __init__(self, embedding_dim, num_heads, forward_expansion, dropout_rate, attn_dropout):
        super().__init__()

        self.MHA = MSA(embedding_dim, num_heads, dropout=attn_dropout, proj_attn=dropout_rate )
        self.MLP = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim*forward_expansion),
            nn.GELU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(embedding_dim*forward_expansion, embedding_dim),
            nn.Dropout(p=dropout_rate),
        )
        self.layer_norm = nn.LayerNorm(embedding_dim, eps=1e-6)
    def forward(self, x):
        # x: (batch_size, patch_num, embedding_dim)
        z = self.MHA(x, x, x) + x 
        output = self.MLP(self.layer_norm(z)) + z 
        return output


#### 3.2.3. **Encoder**

 Bu bölümde Encoder bloğunu oluşturuyoruz. Encoder bloğu ${N}$ adet Encoder Stack bloğundan (${N}$ = num_layers) oluşur.

In [7]:
class Encoder(nn.Module):
    def __init__(self, embedding_dim, num_heads, forward_expansion, num_layers, dropout_rate, attn_dropout):
        super().__init__()
        self.layer_norm = nn.LayerNorm(embedding_dim, eps=1e-6)
        self.encoder_layers = nn.ModuleList([
            EncoderStack(embedding_dim, num_heads, forward_expansion, dropout_rate, attn_dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x):
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(self.layer_norm(x))
        return x

### MLP Head

#### 3.3.1. **Sınıflandırma Başlığı**

Bu blokta Encoder'dan gelen çıktıdan class token'ı çıkarıp bu vektörü sınıflandırma başlığından'den geçireceğiz. Bu başlık basitçe bir lineer katman olacak. Bu katmanın çıktısı 2 boyutlu bir vektör olacak, bu vektöre daha sonra softmax uygulayıp olası sınıfların olasılıklarına ulaşacağız.

In [8]:
class ClassificationHead(nn.Module):
    def __init__(self, embedding_dim, num_classes, dropout_rate):
        super().__init__()
        self.fc = nn.Linear(embedding_dim, num_classes)
        self.layer_norm = nn.LayerNorm(embedding_dim, eps=1e-6)

    def forward(self, x):
        x = self.layer_norm(x)
        x = self.fc(x)
        return x

### Vision Transformer (ViT)

In [9]:
class VisionTransformer(nn.Module):
    def __init__(self, image_channels, image_size, patch_size, embedding_dim, num_heads, forward_expansion, num_layers, num_classes, dropout_rate, attn_dropout):
        super().__init__()
        self.num_classes = num_classes
        self.patch_size = patch_size
        self.embeddings = PatchPosEmbeddings(image_channels, image_size, patch_size, embedding_dim, dropout_rate)
        self.encoder = Encoder( embedding_dim, num_heads, forward_expansion, num_layers, dropout_rate, attn_dropout)
        self.classifier = ClassificationHead(embedding_dim, num_classes, dropout_rate)
        self.emb_dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.embeddings(x)
        x = self.emb_dropout(x)
        x = self.encoder(x)
        class_embd = x[:, 0, :]
        out = self.classifier(class_embd)
        return out
    
    def predict(self, x):
        x = self.forward(x)
        probs = F.softmax(x, dim=-1)
        return torch.argmax(probs, dim=-1)